#### First notebook used for data processing, was used before migrating to jobs

In [58]:
import boto3
import pandas as pd
from io import StringIO
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import json
import numpy as np

In [4]:
BUCKET_NAME = "team1-index-predictor-bucket"

raw_data_filename = "data/raw/sp500.csv"
raw_inference_data_filename = "data/raw/sp500-inference.csv"

s3 = boto3.client("s3")

In [28]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_data_filename)

data = s3_object["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(data))

s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_inference_data_filename)

data = s3_object["Body"].read().decode("utf-8")
inference_df = pd.read_csv(StringIO(data))

df.head(), inference_df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-05-15 09:30:00-04:00,5263.259766,5271.049805,5263.259766,5270.939941,5270.939941,0
1,2024-05-15 09:31:00-04:00,5271.250000,5273.240234,5271.149902,5271.439941,5271.439941,12862437
2,2024-05-15 09:32:00-04:00,5271.779785,5275.759766,5271.779785,5274.950195,5274.950195,11405230
3,2024-05-15 09:33:00-04:00,5275.160156,5277.259766,5273.870117,5276.120117,5276.120117,8963509
4,2024-05-15 09:34:00-04:00,5275.979980,5276.200195,5274.560059,5276.109863,5276.109863,7163548


In [193]:
df = pd.read_csv("../" + raw_data_filename)
inference_df = pd.read_csv("../" + raw_inference_data_filename)
df.head(), inference_df.head()

(                    Datetime         Open         High          Low  \
 0  2024-05-17 09:30:00-04:00  5303.100098  5305.450195  5302.660156   
 1  2024-05-17 09:31:00-04:00  5303.060059  5303.520020  5301.149902   
 2  2024-05-17 09:32:00-04:00  5301.410156  5301.720215  5299.410156   
 3  2024-05-17 09:33:00-04:00  5300.279785  5301.419922  5299.569824   
 4  2024-05-17 09:34:00-04:00  5299.700195  5299.779785  5298.250000   
 
          Close    Adj Close   Volume  
 0  5303.569824  5303.569824        0  
 1  5301.330078  5301.330078  6465998  
 2  5300.509766  5300.509766  6910800  
 3  5299.649902  5299.649902  6913787  
 4  5299.040039  5299.040039  5945542  ,
                     Datetime         Open         High          Low  \
 0  2024-06-14 15:29:00-04:00  5427.899902  5428.529785  5427.779785   
 1  2024-06-14 15:30:00-04:00  5428.410156  5428.470215  5426.439941   
 2  2024-06-14 15:31:00-04:00  5426.609863  5427.770020  5426.339844   
 3  2024-06-14 15:32:00-04:00  5427.3

In [194]:
df = df[["Datetime", "Close", "Open", "High", "Low", "Volume"]]
inference_df = inference_df[["Datetime", "Close", "Open", "High", "Low", "Volume"]]
df.head()

,Datetime,Close,Open,High,Low,Volume
0,2024-05-17 09:30:00-04:00,5303.569824,5303.100098,5305.450195,5302.660156,0
1,2024-05-17 09:31:00-04:00,5301.330078,5303.060059,5303.520020,5301.149902,6465998
2,2024-05-17 09:32:00-04:00,5300.509766,5301.410156,5301.720215,5299.410156,6910800
3,2024-05-17 09:33:00-04:00,5299.649902,5300.279785,5301.419922,5299.569824,6913787
4,2024-05-17 09:34:00-04:00,5299.040039,5299.700195,5299.779785,5298.250000,5945542


In [195]:
# Ensure data is sorted by datetime
df = df.sort_values(by=["Datetime"])
inference_df = inference_df.sort_values(by=["Datetime"])

In [196]:
df["DayOfWeek"] = pd.to_datetime(df["Datetime"]).dt.dayofweek
df["Hour"] = pd.to_datetime(df["Datetime"]).dt.hour
df["Minute"] = pd.to_datetime(df["Datetime"]).dt.minute

inference_df["DayOfWeek"] = pd.to_datetime(inference_df["Datetime"]).dt.dayofweek
inference_df["Hour"] = pd.to_datetime(inference_df["Datetime"]).dt.hour
inference_df["Minute"] = pd.to_datetime(inference_df["Datetime"]).dt.minute

In [197]:
def one_hot_encode_day_of_week(df, max_day_of_week=4):
    for i in range(max_day_of_week + 1):
        df[f"DayOfWeek_{i}"] = (df["DayOfWeek"] == i).astype(int)
    df.drop(columns=["DayOfWeek"], inplace=True)
    return df


one_hot_encode_day_of_week(inference_df)
one_hot_encode_day_of_week(df)

,Datetime,Close,Open,High,Low,Volume,Hour,Minute,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4
0,2024-05-17 09:30:00-04:00,5303.569824,5303.100098,5305.450195,5302.660156,0,9,30,0,0,0,0,1
1,2024-05-17 09:31:00-04:00,5301.330078,5303.060059,5303.520020,5301.149902,6465998,9,31,0,0,0,0,1
2,2024-05-17 09:32:00-04:00,5300.509766,5301.410156,5301.720215,5299.410156,6910800,9,32,0,0,0,0,1
3,2024-05-17 09:33:00-04:00,5299.649902,5300.279785,5301.419922,5299.569824,6913787,9,33,0,0,0,0,1
4,2024-05-17 09:34:00-04:00,5299.040039,5299.700195,5299.779785,5298.250000,5945542,9,34,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7711,2024-06-14 15:55:00-04:00,5425.430176,5424.959961,5425.430176,5424.169922,17406000,15,55,0,0,0,0,1
7712,2024-06-14 15:56:00-04:00,5427.629883,5425.450195,5427.830078,5425.370117,18678000,15,56,0,0,0,0,1
7713,2024-06-14 15:57:00-04:00,5427.450195,5427.589844,5427.759766,5426.620117,20324000,15,57,0,0,0,0,1
7714,2024-06-14 15:58:00-04:00,5429.200195,5427.430176,5429.200195,5427.430176,26459000,15,58,0,0,0,0,1


In [198]:
df = df.drop(columns=["Datetime"])
inference_df = inference_df.drop(columns=["Datetime"])

In [199]:
total_size = len(df)
# test_size = int(0.1 * total_size)
# validation_size = int(0.1 * total_size)
test_size = 390
validation_size = 390
train_size = total_size - test_size - validation_size

test_start_idx = total_size - test_size
validation_start_idx = test_start_idx - validation_size

train_df = df.iloc[:validation_start_idx]
validation_df = df.iloc[validation_start_idx:test_start_idx]
test_df = df.iloc[test_start_idx:]

In [200]:
print("Training set size:", len(train_df))
print("Validation set size:", len(validation_df))
print("Test set size:", len(test_df))
print("Inference data size:", len(inference_df))

Training set size: 6936
Validation set size: 390
Test set size: 390
Inference data size: 31


In [201]:
# No need to scale for XGBoost
# scaler = MinMaxScaler()

# train_series = scaler.fit_transform(train_df)
# validation_series = scaler.transform(validation_df)
# test_series = scaler.transform(test_df)
# inference_series = scaler.transform(inference_df)

In [202]:
# store later as metadata in feature store
# if scaler is MinMaxScaler
# if type(scaler) == MinMaxScaler:
#     scaler_params = {
#         "data_min": scaler.data_min_[0],
#         "data_max": scaler.data_max_[0],
#         "data_range": scaler.data_range_[0],
#         "min_": scaler.min_[0],
#         "scale_": scaler.scale_[0],
#     }
# elif type(scaler) == StandardScaler:
#     scaler_params = {
#         "mean": scaler.mean_[0],
#         "variance": scaler.var_[0],
#         "scale_": scaler.scale_[0],
#     }
# else:
#     raise ValueError("Unsupported scaler type")
# scaler_params

In [203]:
def prepare_data(df, lag):
    """Lag last lag values of Open, Close, High, Low, Volume as features, and target is whether
    the Close price will be higher than the current price in 3 minutes."""

    df = df.copy()
    lagged_data = {}

    for i in range(1, lag + 1):
        for col in ["Open", "Close", "High", "Low", "Volume"]:
            lagged_data[f"{col}_lag_{i}"] = df[col].shift(i)

    lagged_df = pd.DataFrame(lagged_data)
    df = pd.concat([df, lagged_df], axis=1)

    df["Close_target"] = (df["Close"].shift(-3) > df["Close"]).astype(int)

    df.dropna(inplace=True)

    return df

In [204]:
lag = 30

train_df = prepare_data(train_df, lag)
validation_df = prepare_data(validation_df, lag)
test_df = prepare_data(test_df, lag)
inference_df = prepare_data(inference_df, lag)

In [132]:
%mkdir -p ../data/processed

In [206]:
train_df.to_csv("../data/processed/train.csv", index=False)
validation_df.to_csv("../data/processed/validation.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)
inference_df.to_csv("../data/processed/inference.csv", index=False)

In [53]:
# REVISIT: use feature store later instead of plain s3, version things and store
# scaler params as metadata in feature store


def upload_df_to_s3(df, bucket_name, object_name):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=csv_buffer.getvalue())


root_folder = "data/processed"

upload_df_to_s3(train_df, BUCKET_NAME, f"{root_folder}/train.csv")
upload_df_to_s3(validation_df, BUCKET_NAME, f"{root_folder}/validation.csv")
upload_df_to_s3(test_df, BUCKET_NAME, f"{root_folder}/test.csv")
upload_df_to_s3(inference_df, BUCKET_NAME, f"{root_folder}/inference.csv")

In [134]:
scaler_params_json = json.dumps(scaler_params)

In [135]:
with open("../data/processed/scaler_params.json", "w") as f:
    f.write(scaler_params_json)

In [59]:
s3.put_object(
    Bucket=BUCKET_NAME, Key=f"{root_folder}/scaler_params.json", Body=scaler_params_json
)

{'ResponseMetadata': {'RequestId': 'ZQF23Z0WFZ3B9GKG',
  'HostId': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
   'x-amz-request-id': 'ZQF23Z0WFZ3B9GKG',
   'date': 'Fri, 14 Jun 2024 10:00:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
 'ServerSideEncryption': 'AES256'}

Only to test that upload worked

In [55]:
s3.list_objects(Bucket=BUCKET_NAME)["Contents"]

[{'Key': 'data/processed/test.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"4d1370a95a9260cfba866dcc4f51791c"',
  'Size': 36263,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/train.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"1e087f7f9658fc8c539bcfb3ce575d20"',
  'Size': 292145,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/validation.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"9767fe903678ae5cb4bf72847b456203"',
  'Size': 36364,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/raw/sp500.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 7, 19, 40, tzinfo=

In [60]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=f"{root_folder}/train.csv")

loaded_data = s3_object["Body"].read().decode("utf-8")
loaded_df = pd.read_csv(StringIO(loaded_data))
loaded_df.head()

,Datetime,Close
0,2024-05-15 09:30:00-04:00,0.429912
1,2024-05-15 09:31:00-04:00,0.432670
2,2024-05-15 09:32:00-04:00,0.452035
3,2024-05-15 09:33:00-04:00,0.458489
4,2024-05-15 09:34:00-04:00,0.458433


In [58]:
import boto3
import pandas as pd
from io import StringIO
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import json
import numpy as np

In [4]:
BUCKET_NAME = "team1-index-predictor-bucket"

raw_data_filename = "data/raw/sp500.csv"
raw_inference_data_filename = "data/raw/sp500-inference.csv"

s3 = boto3.client("s3")

In [28]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_data_filename)

data = s3_object["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(data))

s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_inference_data_filename)

data = s3_object["Body"].read().decode("utf-8")
inference_df = pd.read_csv(StringIO(data))

df.head(), inference_df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-05-15 09:30:00-04:00,5263.259766,5271.049805,5263.259766,5270.939941,5270.939941,0
1,2024-05-15 09:31:00-04:00,5271.250000,5273.240234,5271.149902,5271.439941,5271.439941,12862437
2,2024-05-15 09:32:00-04:00,5271.779785,5275.759766,5271.779785,5274.950195,5274.950195,11405230
3,2024-05-15 09:33:00-04:00,5275.160156,5277.259766,5273.870117,5276.120117,5276.120117,8963509
4,2024-05-15 09:34:00-04:00,5275.979980,5276.200195,5274.560059,5276.109863,5276.109863,7163548


In [193]:
df = pd.read_csv("../" + raw_data_filename)
inference_df = pd.read_csv("../" + raw_inference_data_filename)
df.head(), inference_df.head()

(                    Datetime         Open         High          Low  \
 0  2024-05-17 09:30:00-04:00  5303.100098  5305.450195  5302.660156   
 1  2024-05-17 09:31:00-04:00  5303.060059  5303.520020  5301.149902   
 2  2024-05-17 09:32:00-04:00  5301.410156  5301.720215  5299.410156   
 3  2024-05-17 09:33:00-04:00  5300.279785  5301.419922  5299.569824   
 4  2024-05-17 09:34:00-04:00  5299.700195  5299.779785  5298.250000   
 
          Close    Adj Close   Volume  
 0  5303.569824  5303.569824        0  
 1  5301.330078  5301.330078  6465998  
 2  5300.509766  5300.509766  6910800  
 3  5299.649902  5299.649902  6913787  
 4  5299.040039  5299.040039  5945542  ,
                     Datetime         Open         High          Low  \
 0  2024-06-14 15:29:00-04:00  5427.899902  5428.529785  5427.779785   
 1  2024-06-14 15:30:00-04:00  5428.410156  5428.470215  5426.439941   
 2  2024-06-14 15:31:00-04:00  5426.609863  5427.770020  5426.339844   
 3  2024-06-14 15:32:00-04:00  5427.3

In [194]:
df = df[["Datetime", "Close", "Open", "High", "Low", "Volume"]]
inference_df = inference_df[["Datetime", "Close", "Open", "High", "Low", "Volume"]]
df.head()

,Datetime,Close,Open,High,Low,Volume
0,2024-05-17 09:30:00-04:00,5303.569824,5303.100098,5305.450195,5302.660156,0
1,2024-05-17 09:31:00-04:00,5301.330078,5303.060059,5303.520020,5301.149902,6465998
2,2024-05-17 09:32:00-04:00,5300.509766,5301.410156,5301.720215,5299.410156,6910800
3,2024-05-17 09:33:00-04:00,5299.649902,5300.279785,5301.419922,5299.569824,6913787
4,2024-05-17 09:34:00-04:00,5299.040039,5299.700195,5299.779785,5298.250000,5945542


In [195]:
# Ensure data is sorted by datetime
df = df.sort_values(by=["Datetime"])
inference_df = inference_df.sort_values(by=["Datetime"])

In [196]:
df["DayOfWeek"] = pd.to_datetime(df["Datetime"]).dt.dayofweek
df["Hour"] = pd.to_datetime(df["Datetime"]).dt.hour
df["Minute"] = pd.to_datetime(df["Datetime"]).dt.minute

inference_df["DayOfWeek"] = pd.to_datetime(inference_df["Datetime"]).dt.dayofweek
inference_df["Hour"] = pd.to_datetime(inference_df["Datetime"]).dt.hour
inference_df["Minute"] = pd.to_datetime(inference_df["Datetime"]).dt.minute

In [197]:
def one_hot_encode_day_of_week(df, max_day_of_week=4):
    for i in range(max_day_of_week + 1):
        df[f"DayOfWeek_{i}"] = (df["DayOfWeek"] == i).astype(int)
    df.drop(columns=["DayOfWeek"], inplace=True)
    return df


one_hot_encode_day_of_week(inference_df)
one_hot_encode_day_of_week(df)

,Datetime,Close,Open,High,Low,Volume,Hour,Minute,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4
0,2024-05-17 09:30:00-04:00,5303.569824,5303.100098,5305.450195,5302.660156,0,9,30,0,0,0,0,1
1,2024-05-17 09:31:00-04:00,5301.330078,5303.060059,5303.520020,5301.149902,6465998,9,31,0,0,0,0,1
2,2024-05-17 09:32:00-04:00,5300.509766,5301.410156,5301.720215,5299.410156,6910800,9,32,0,0,0,0,1
3,2024-05-17 09:33:00-04:00,5299.649902,5300.279785,5301.419922,5299.569824,6913787,9,33,0,0,0,0,1
4,2024-05-17 09:34:00-04:00,5299.040039,5299.700195,5299.779785,5298.250000,5945542,9,34,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7711,2024-06-14 15:55:00-04:00,5425.430176,5424.959961,5425.430176,5424.169922,17406000,15,55,0,0,0,0,1
7712,2024-06-14 15:56:00-04:00,5427.629883,5425.450195,5427.830078,5425.370117,18678000,15,56,0,0,0,0,1
7713,2024-06-14 15:57:00-04:00,5427.450195,5427.589844,5427.759766,5426.620117,20324000,15,57,0,0,0,0,1
7714,2024-06-14 15:58:00-04:00,5429.200195,5427.430176,5429.200195,5427.430176,26459000,15,58,0,0,0,0,1


In [198]:
df = df.drop(columns=["Datetime"])
inference_df = inference_df.drop(columns=["Datetime"])

In [199]:
total_size = len(df)
# test_size = int(0.1 * total_size)
# validation_size = int(0.1 * total_size)
test_size = 390
validation_size = 390
train_size = total_size - test_size - validation_size

test_start_idx = total_size - test_size
validation_start_idx = test_start_idx - validation_size

train_df = df.iloc[:validation_start_idx]
validation_df = df.iloc[validation_start_idx:test_start_idx]
test_df = df.iloc[test_start_idx:]

In [200]:
print("Training set size:", len(train_df))
print("Validation set size:", len(validation_df))
print("Test set size:", len(test_df))
print("Inference data size:", len(inference_df))

Training set size: 6936
Validation set size: 390
Test set size: 390
Inference data size: 31


In [201]:
# No need to scale for XGBoost
# scaler = MinMaxScaler()

# train_series = scaler.fit_transform(train_df)
# validation_series = scaler.transform(validation_df)
# test_series = scaler.transform(test_df)
# inference_series = scaler.transform(inference_df)

In [202]:
# store later as metadata in feature store
# if scaler is MinMaxScaler
# if type(scaler) == MinMaxScaler:
#     scaler_params = {
#         "data_min": scaler.data_min_[0],
#         "data_max": scaler.data_max_[0],
#         "data_range": scaler.data_range_[0],
#         "min_": scaler.min_[0],
#         "scale_": scaler.scale_[0],
#     }
# elif type(scaler) == StandardScaler:
#     scaler_params = {
#         "mean": scaler.mean_[0],
#         "variance": scaler.var_[0],
#         "scale_": scaler.scale_[0],
#     }
# else:
#     raise ValueError("Unsupported scaler type")
# scaler_params

In [203]:
def prepare_data(df, lag):
    """Lag last lag values of Open, Close, High, Low, Volume as features, and target is whether
    the Close price will be higher than the current price in 3 minutes."""

    df = df.copy()
    lagged_data = {}

    for i in range(1, lag + 1):
        for col in ["Open", "Close", "High", "Low", "Volume"]:
            lagged_data[f"{col}_lag_{i}"] = df[col].shift(i)

    lagged_df = pd.DataFrame(lagged_data)
    df = pd.concat([df, lagged_df], axis=1)

    df["Close_target"] = (df["Close"].shift(-3) > df["Close"]).astype(int)

    df.dropna(inplace=True)

    return df

In [204]:
lag = 30

train_df = prepare_data(train_df, lag)
validation_df = prepare_data(validation_df, lag)
test_df = prepare_data(test_df, lag)
inference_df = prepare_data(inference_df, lag)

In [132]:
%mkdir -p ../data/processed

In [206]:
train_df.to_csv("../data/processed/train.csv", index=False)
validation_df.to_csv("../data/processed/validation.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)
inference_df.to_csv("../data/processed/inference.csv", index=False)

In [53]:
# REVISIT: use feature store later instead of plain s3, version things and store
# scaler params as metadata in feature store


def upload_df_to_s3(df, bucket_name, object_name):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=csv_buffer.getvalue())


root_folder = "data/processed"

upload_df_to_s3(train_df, BUCKET_NAME, f"{root_folder}/train.csv")
upload_df_to_s3(validation_df, BUCKET_NAME, f"{root_folder}/validation.csv")
upload_df_to_s3(test_df, BUCKET_NAME, f"{root_folder}/test.csv")
upload_df_to_s3(inference_df, BUCKET_NAME, f"{root_folder}/inference.csv")

In [134]:
scaler_params_json = json.dumps(scaler_params)

In [135]:
with open("../data/processed/scaler_params.json", "w") as f:
    f.write(scaler_params_json)

In [59]:
s3.put_object(
    Bucket=BUCKET_NAME, Key=f"{root_folder}/scaler_params.json", Body=scaler_params_json
)

{'ResponseMetadata': {'RequestId': 'ZQF23Z0WFZ3B9GKG',
  'HostId': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
   'x-amz-request-id': 'ZQF23Z0WFZ3B9GKG',
   'date': 'Fri, 14 Jun 2024 10:00:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
 'ServerSideEncryption': 'AES256'}

Only to test that upload worked

In [55]:
s3.list_objects(Bucket=BUCKET_NAME)["Contents"]

[{'Key': 'data/processed/test.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"4d1370a95a9260cfba866dcc4f51791c"',
  'Size': 36263,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/train.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"1e087f7f9658fc8c539bcfb3ce575d20"',
  'Size': 292145,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/validation.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"9767fe903678ae5cb4bf72847b456203"',
  'Size': 36364,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/raw/sp500.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 7, 19, 40, tzinfo=

In [60]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=f"{root_folder}/train.csv")

loaded_data = s3_object["Body"].read().decode("utf-8")
loaded_df = pd.read_csv(StringIO(loaded_data))
loaded_df.head()

,Datetime,Close
0,2024-05-15 09:30:00-04:00,0.429912
1,2024-05-15 09:31:00-04:00,0.432670
2,2024-05-15 09:32:00-04:00,0.452035
3,2024-05-15 09:33:00-04:00,0.458489
4,2024-05-15 09:34:00-04:00,0.458433
